<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction</a></span></li><li><span><a href="#Imports" data-toc-modified-id="Imports-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Imports</a></span></li><li><span><a href="#Load-the-dataset" data-toc-modified-id="Load-the-dataset-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Load the dataset</a></span></li><li><span><a href="#Dataset-splitting:-train,-val-and-test-sets" data-toc-modified-id="Dataset-splitting:-train,-val-and-test-sets-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Dataset splitting: train, val and test sets</a></span></li><li><span><a href="#Pre-trained-Tokenizer" data-toc-modified-id="Pre-trained-Tokenizer-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Pre-trained Tokenizer</a></span></li><li><span><a href="#Create-a-specific-tokenizer-for-the-dataset" data-toc-modified-id="Create-a-specific-tokenizer-for-the-dataset-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Create a specific tokenizer for the dataset</a></span></li><li><span><a href="#Model-FineTuning" data-toc-modified-id="Model-FineTuning-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Model FineTuning</a></span></li><li><span><a href="#Inference" data-toc-modified-id="Inference-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Inference</a></span></li><li><span><a href="#Folder-clean-up" data-toc-modified-id="Folder-clean-up-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Folder clean-up</a></span></li><li><span><a href="#References" data-toc-modified-id="References-10"><span class="toc-item-num">10&nbsp;&nbsp;</span>References</a></span></li></ul></div>

# Introduction
<hr style="border:2px solid black"> </hr>

<div class="alert alert-block alert-warning">
<font color=black>

**What?** Fine tuning pre-trained BERT model on Text Classification Task

</font>
</div>

In [22]:
!pip install transformers
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Imports
<hr style="border:2px solid black"> </hr>

In [23]:
# this is a test

import os
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from transformers import AutoTokenizer
from datasets import load_dataset, DatasetDict, Dataset
from transformers import (
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding
)
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
import sklearn.metrics as metrics

# Load the dataset
<hr style="border:2px solid black"> </hr>

<div class="alert alert-block alert-info">
<font color=black>


- Process of finetuning a pre-trained BERT model towards a text classification task, more specificially, the Quora Question Pairs.
- The goal of this competition is to predict which of the provided pairs of questions contain two questions with the same meaning. The ground truth is the set of labels that have been supplied by human experts. The ground truth labels are inherently subjective, as the true meaning of sentences can never be known with certainty. Human labeling is also a 'noisy' process, and reasonable people will disagree. As a result, the ground truth labels on this dataset should be taken to be 'informed' but not 100% accurate, and may include incorrect labeling. 

</font>
</div>

In [24]:
dataset_dict = load_dataset("quora")
dataset_dict #= dataset_dict[:10000]

  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['questions', 'is_duplicate'],
        num_rows: 404290
    })
})

In [25]:
dataset_dict['train'][0]

{'questions': {'id': [1, 2],
  'text': ['What is the step by step guide to invest in share market in india?',
   'What is the step by step guide to invest in share market?']},
 'is_duplicate': False}

# Dataset splitting: train, val and test sets
<hr style="border:2px solid black"> </hr>

In [26]:
test_size = 0.1
val_size = 0.1
dataset_dict_test = dataset_dict['train'].train_test_split(test_size=test_size)
dataset_dict_train_val = dataset_dict_test['train'].train_test_split(test_size=val_size)

dataset_dict = DatasetDict({
    "train": dataset_dict_train_val["train"],
    "val": dataset_dict_train_val["test"],
    "test": dataset_dict_test["test"]
})
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['questions', 'is_duplicate'],
        num_rows: 327474
    })
    val: Dataset({
        features: ['questions', 'is_duplicate'],
        num_rows: 36387
    })
    test: Dataset({
        features: ['questions', 'is_duplicate'],
        num_rows: 40429
    })
})

In [27]:
dir(dataset_dict)

['__class__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_check_values_features',
 '_check_values_type',
 'align_labels_with_mapping',
 'cache_files',
 'cast',
 'cast_column',
 'class_encode_column',
 'cleanup_cache_files',
 'clear',
 'column_names',
 'copy',
 'data',
 'filter',
 'flatten',
 'formatted_as',
 'from_csv',
 'from_json',
 'from_parquet',
 'from_text',
 'fromkeys',
 'get',
 'items',
 'keys',
 'load_from_disk',
 'map',
 'num_columns',
 'num_rows',
 'pop',
 'popitem',
 'prepare_for_task',
 'push_to_hub',
 'remove_columns',
 'rename_column',
 'rename_columns',
 '

# Pre-trained Tokenizer
<hr style="border:2px solid black"> </hr>

<div class="alert alert-block alert-info">
<font color=black>
    
- We'll use a pre-trained tokenizer available from the huggingface model repository.
- This can be found [here](https://huggingface.co/transformers/model_doc/distilbert.html).  

</font>
</div>

In [28]:
pretrained_model_name_or_path = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path)
tokenizer

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

PreTrainedTokenizerFast(name_or_path='distilbert-base-uncased', vocab_size=30522, model_max_len=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [29]:
# Let's feed the tokenizer with a pair of sentences.
encoded_input = tokenizer(
    'What is the step by step guide to invest in share market in india?',
    'What is the step by step guide to invest in share market?'
)
encoded_input

{'input_ids': [101, 2054, 2003, 1996, 3357, 2011, 3357, 5009, 2000, 15697, 1999, 3745, 3006, 1999, 2634, 1029, 102, 2054, 2003, 1996, 3357, 2011, 3357, 5009, 2000, 15697, 1999, 3745, 3006, 1029, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

Decoding the tokenized inputs, this model's tokenizer adds some special tokens such as, `[SEP]`, that is used to indicate which token belongs to which segment/pair.

In [30]:
tokenizer.decode(encoded_input["input_ids"])

'[CLS] what is the step by step guide to invest in share market in india? [SEP] what is the step by step guide to invest in share market? [SEP]'

# Create a specific tokenizer for the dataset
<hr style="border:2px solid black"> </hr>

In [31]:
def tokenize_fn(examples):
    """
    The proprocessing step will be task specific 
    and database specific
    """

    labels = [int(label) for label in examples['is_duplicate']]
    texts = [question['text'] for question in examples['questions']]
    texts1 = [text[0] for text in texts]
    texts2 = [text[1] for text in texts]
    tokenized_examples = tokenizer(texts1, texts2)
    tokenized_examples['labels'] = labels
    return tokenized_examples

In [32]:
dataset_dict_tokenized = dataset_dict.map(
    tokenize_fn,
    batched=True,
    num_proc=8,
    remove_columns=['is_duplicate', 'questions']
)
dataset_dict_tokenized

#1:   0%|          | 0/41 [00:00<?, ?ba/s]

#0:   0%|          | 0/41 [00:00<?, ?ba/s]

#3:   0%|          | 0/41 [00:00<?, ?ba/s]

#2:   0%|          | 0/41 [00:00<?, ?ba/s]

#5:   0%|          | 0/41 [00:00<?, ?ba/s]

...

#6:   0%|          | 0/6 [00:00<?, ?ba/s]

#4:   0%|          | 0/6 [00:00<?, ?ba/s]

#2:   0%|          | 0/6 [00:00<?, ?ba/s]

#7:   0%|          | 0/6 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 327474
    })
    val: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 36387
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 40429
    })
})

In [33]:
# Let's look at the first entry
dataset_dict_tokenized['train'][0]

{'input_ids': [101,
  2339,
  3475,
  1005,
  1056,
  5264,
  1037,
  2112,
  1997,
  6027,
  1029,
  102,
  4855,
  2026,
  16568,
  14012,
  1010,
  2054,
  1005,
  1055,
  1037,
  4189,
  14255,
  19170,
  1029,
  102],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 'labels': 0}

# Model FineTuning
<hr style="border:2px solid black"> </hr>

<div class="alert alert-block alert-info">
<font color=black>
    
- Having preprocessed our raw dataset, for our text classification task, we use `AutoModelForSequenceClassification` class to load the pre-trained model, the only other argument we need to specify is the number of class/label our text classification task has. 
- The following warning will be displayed "You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference, and this is expected.
- You will have to run this on some GPU as it would too long for a CPU-based training.

</font>
</div>

In [34]:
model_checkpoint = 'text_classification'
num_labels = 2

In [35]:
if os.path.isdir(model_checkpoint):
    # if present read from last checkpoint
    model = AutoModelForSequenceClassification.from_pretrained(
        model_checkpoint)
else:
    # If not present start training
    model = AutoModelForSequenceClassification.from_pretrained(
        pretrained_model_name_or_path, num_labels=num_labels)

print('# of parameters: ', model.num_parameters())
model

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier

# of parameters:  66955010


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
       

In [36]:
data_collator = DataCollatorWithPadding(tokenizer, padding=True)
data_collator

DataCollatorWithPadding(tokenizer=PreTrainedTokenizerFast(name_or_path='distilbert-base-uncased', vocab_size=30522, model_max_len=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}), padding=True, max_length=None, pad_to_multiple_of=None, return_tensors='pt')

In [43]:
batch_size = 64
args = TrainingArguments(
    "quora",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=2,
    weight_decay=0.01,
    # https://github.com/huggingface/transformers/issues/14051
    load_best_model_at_end=False
)

trainer = Trainer(
    model,
    args,
    data_collator=data_collator,
    train_dataset=dataset_dict_tokenized["train"],
    eval_dataset=dataset_dict_tokenized['val']
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [44]:
if not os.path.isdir(model_checkpoint):
    trainer.train()
    model.save_pretrained(model_checkpoint)

***** Running training *****
  Num examples = 327474
  Num Epochs = 2
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 10234
  Number of trainable parameters = 66955010


Epoch,Training Loss,Validation Loss


Saving model checkpoint to quora/checkpoint-500
Configuration saved in quora/checkpoint-500/config.json
Model weights saved in quora/checkpoint-500/pytorch_model.bin


KeyboardInterrupt: ignored

# Inference
<hr style="border:2px solid black"> </hr>

The next couple of code chunks performs batch inferencing on our dataset, and reports standard binary classification evaluation metrics.

In [45]:
def predict(model, example, round_digits: int = 5):
    input_ids = example['input_ids'].to(model.device)
    attention_mask = example['attention_mask'].to(model.device)
    batch_labels = example['labels'].detach().cpu().numpy().tolist()
    model.eval()
    with torch.no_grad():
        batch_output = model(input_ids, attention_mask)

    batch_scores = F.softmax(batch_output.logits, dim=-1)[:, 1]
    batch_scores = np.round(
        batch_scores.detach().cpu().numpy(), round_digits).tolist()
    return batch_scores, batch_labels

In [46]:
def predict_data_loader(model, data_loader: DataLoader) -> pd.DataFrame:
    scores = []
    labels = []
    for example in data_loader:
        batch_scores, batch_labels = predict(model, example)
        scores += batch_scores
        labels += batch_labels

    df_predictions = pd.DataFrame.from_dict(
        {'scores': scores, 'labels': labels})
    return df_predictions

In [47]:
data_loader = DataLoader(
    dataset_dict_tokenized['test'], collate_fn=data_collator, batch_size=64)
start = time.time()
df_predictions = predict_data_loader(model, data_loader)
end = time.time()
print('elapsed: ', end - start)
print(df_predictions.shape)
df_predictions.head()

elapsed:  86.21714472770691
(40429, 2)


,scores,labels
0,0.89403,1
1,0.00874,0
2,0.39959,0
3,0.81734,1
4,0.39334,0


In [50]:
def compute_binary_classification_metrics(y_true, y_score, round_digits: int = 3):
    auc = metrics.roc_auc_score(y_true, y_score)
    log_loss = metrics.log_loss(y_true, y_score)

    precision, recall, threshold = metrics.precision_recall_curve(
        y_true, y_score)
    f1 = 2 * (precision * recall) / (precision + recall)

    mask = ~np.isnan(f1)
    f1 = f1[mask]
    precision = precision[mask]
    recall = recall[mask]

    best_index = np.argmax(f1)
    precision = precision[best_index]
    recall = recall[best_index]
    f1 = f1[best_index]
    return {
        'auc': round(auc, round_digits),
        'precision': round(precision, round_digits),
        'recall': round(recall, round_digits),
        'f1': round(f1, round_digits),
        'log_loss': round(log_loss, round_digits)
    }

In [51]:
result = compute_binary_classification_metrics(df_predictions['labels'], df_predictions['scores'])

In [52]:
result

{'auc': 0.913,
 'precision': 0.72,
 'recall': 0.867,
 'f1': 0.787,
 'log_loss': 0.357}

# Folder clean-up
<hr style="border:2px solid black"> </hr>

In [53]:
!ls

quora  sample_data


In [ ]:
!rm -rf quora

# References
<hr style="border:2px solid black"> </hr>

<div class="alert alert-block alert-warning">
<font color=black>
    
- [Quora Question Pairs](https://www.kaggle.com/c/quora-question-pairs/data)    
- [Jupyter Notebook: Fine-tuning a model on a text classification task](https://nbviewer.org/github/huggingface/notebooks/blob/main/examples/text_classification.ipynb)
- [Blog: Faster and smaller quantized NLP with Hugging Face and ONNX Runtime](https://medium.com/microsoftazure/faster-and-smaller-quantized-nlp-with-hugging-face-and-onnx-runtime-ec5525473bb7)
- [PyTorch Documentation: Dynamic Quantization](https://pytorch.org/tutorials/recipes/recipes/dynamic_quantization.html)
- [Finetuning Pre-trained BERT Model on Text Classification Task](https://github.com/ethen8181/machine-learning/blob/master/model_deployment/transformers/text_classification_onnxruntime.ipynb)

</font>
</div>